In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import data
from skimage import color
import scipy.sparse
import scipy.sparse.linalg
import random
import sys

In [3]:
data = pd.read_csv('./data.csv')
matrix = np.array(data)

In [3]:
def filter_data(images, labels, digits, negate):
    mask = np.zeros(len(labels)).astype(bool)
    for digit in digits:
        mask += mask | labels == digit
    return (images[:,~mask], labels[~mask]) if not negate else (images[:,mask], labels[mask])

In [10]:
def SGD(l, grad_l, w0, data, batch_size, n_epoch, alpha):
    X, Y = data
    d, N = X.shape
    epoch_iter = int(N / batch_size)
    w = w0
    X_copy = np.copy(X)
    Y_copy = np.copy(Y)
    for epoch in range(n_epoch):
        # print(epoch)
        for epoch in range(epoch_iter):
            d, N = X.shape
            indices = np.arange(N)
            batch_idx = np.random.choice(indices, batch_size)
            mask = np.zeros((N, ), dtype=bool)
            mask[batch_idx] = True
            x_batch = X[:, mask]
            y_batch = Y[mask]
            grad = grad_l((w, x_batch, y_batch))
            w = w - alpha * grad.T
            X = X[:, ~mask]
            Y = Y[~mask]
        X = np.copy(X_copy)
        Y = np.copy(Y_copy)
    return w

In [4]:
def fw(w, x):
    return sigmoid(x.T @ w)

def sigmoid(z): 
    sig = 1 / (1 + np.exp(-z))
    return sig

def grad_l(inputs):
    w = inputs[0]
    x = inputs[1]
    y = inputs[2]
    N = np.size(y)
    sigmoid_value = sigmoid(x.T @ w)
    return 1/N * (2 * (sigmoid_value - y).T @ (sigmoid_value * (1 - sigmoid_value) * x.T))

In [7]:
def getHat(X, w):
    X_hat = np.concatenate((np.ones((1,X.shape[1])), X), axis = 0)
    w_hat = np.concatenate((np.ones((1,w.shape[1])), w), axis = 0)
    return (X_hat, w_hat)

def split_data(X, Y, Ntrain):
    d, N = X.shape
    idx = np.arange(N)
    np.random.shuffle(idx)
    train_idx = idx[:Ntrain]
    test_idx = idx[Ntrain:]
    Xtrain = X[:, train_idx]
    Ytrain = Y[train_idx]
    
    Xtest = X[:, test_idx]
    Ytest = Y[test_idx]

    return (Xtrain, Ytrain), (Xtest, Ytest)

In [91]:
digits = [0,1]

X = (matrix[:,1:]).T
Y = matrix[:,0]

X, Y = filter_data(X, Y, digits, True)

d, N = X.shape
X = X/255
w_start = np.zeros((d,1))

epoch = 10
batch_size = 100

X, w_start = getHat(X, w_start)
Y = Y.reshape((N,1))
sgd = SGD(fw, grad_l, w_start, (X, Y), batch_size, epoch, 1)

In [11]:
from sklearn.datasets import make_classification

# Load data
X, Y = make_classification(n_samples=500, n_features=2, n_redundant=0, n_informative=1, 
                            n_clusters_per_class=1)
X = X.T # To make it d x N

# Check the shape
print(f"Shape of X: {X.shape}")
print(f"Shape of Y: {Y.shape}")

# Memorize the shape
d, N = X.shape

epoch = 10
batch_size = 100

# Add dimension on Y
# Y = Y.reshape((N, 1))
n_points = int(len(Y) * 8 / 10)
(Xtrain, Ytrain), (Xtest, Ytest) = split_data(X, Y, n_points)

w_start = np.zeros((d,1))
X_hat, w_start = getHat(Xtrain, w_start)
sgd = SGD(fw, grad_l, w_start, (X_hat, Ytrain), batch_size, epoch, 1)
sgd_calc = lambda x: -sgd[1] / sgd[2] * x - sgd[0] / sgd[2]  
x = np.linspace(X.min(), X.max(), 1000)
y = [sgd_calc(xi) for xi in x]

Shape of X: (2, 500)
Shape of Y: (500,)


In [16]:
I0 = (Ytest==0)
I1 = (Ytest==1)

Ytest0 = Ytest[I0]
Ytest1 = Ytest[I1]
Xtest0 = Xtest[:, I0]
Xtest1 = Xtest[:, I1]

plt.scatter(Xtest0[0,:],Xtest0[1,:], c = Ytest0)
plt.plot(x,y)
plt.show()

plt.scatter(Xtest1[0,:],Xtest1[1,:], c = Ytest1)
plt.plot(x,y)
plt.show()

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [27]:
def classificator(w, x, treshold = 0.5):
    return sigmoid(x.T @ w) >= treshold

In [29]:
correct_y = []
for i in range(np.size(Ytest)):
    x = Xtest[:,i]
    x = np.insert(x,0,1)
    y = Ytest[i]
    classification = classificator(sgd, x)
    y_hat = 1 if classificator(sgd, x) else 0
    if y == y_hat:
        correct_y.append(i)
print(len(correct_y) / np.size(Y) * 100)

19.400000000000002
